<a href="https://colab.research.google.com/github/betegon/16-QAM-Detection-MIMO/blob/master/TAC_MIMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import numpy as np


## EJEMPLO CON 2 antenas


In [0]:
k = 2 # 2 antennas

In [0]:
s_matrix = np.array([[-3, -3],[-3, 3]])
print(s_matrix)
s = (np.array([s_matrix.flatten('F')])).transpose()
print("s\n",s)
t = s**2
print(t)

[[-3 -3]
 [-3  3]]
s
 [[-3]
 [-3]
 [-3]
 [ 3]]
[[9]
 [9]
 [9]
 [9]]


In [0]:
# CREACIÓN DE H 

k = 2  # number of antennas
# H is CN(0,1)
mu = np.zeros(2*k)
sigma = np.ones((k,k*2))


H = (1/np.sqrt(2))*np.random.normal(loc=mu, scale=sigma, size=(k,k*2)).view(np.complex128) 
print(H)
a = np.column_stack((np.real(H), -np.imag(H)))
print("Fila1: \n",a,"\n")
b = np.column_stack((np.imag(H), np.real(H)))
print("Fila2\n",b,"\n")
H_expanded = np.concatenate((a,b), axis=0)
print("Matriz H expandida\n",H_expanded)

[[ 0.66214108-0.79578875j -0.45171898-0.24642896j]
 [ 0.38983823-0.29155658j  1.23646399-0.51144257j]]
Fila1: 
 [[ 0.66214108 -0.45171898  0.79578875  0.24642896]
 [ 0.38983823  1.23646399  0.29155658  0.51144257]] 

Fila2
 [[-0.79578875 -0.24642896  0.66214108 -0.45171898]
 [-0.29155658 -0.51144257  0.38983823  1.23646399]] 

Matriz H expandida
 [[ 0.66214108 -0.45171898  0.79578875  0.24642896]
 [ 0.38983823  1.23646399  0.29155658  0.51144257]
 [-0.79578875 -0.24642896  0.66214108 -0.45171898]
 [-0.29155658 -0.51144257  0.38983823  1.23646399]]


In [0]:
def awgn(signal, snr, seed=False):
    """ Generate Aditive White Gaussian Noise (AWGN) from an input signal.

    Args:
        signal  (np.array): Input signal. e.g. signal = np.array([[1, 2.3, -1]])
        snr        (float): Signal-to-noise ratio in dB.
        seed        (bool): Set seed to reproduce results (good for debugging).
                            Defaults to False.

    Returns:
        noise (np.array): AWGN generated from an input signal.

    """
    if seed: np.random.seed(1)
    sigpower = sum([ math.pow(abs(signal[i]),2) for i in range(len(signal)) ])
    sigpower = sigpower / len(signal)
    noisepower = sigpower / (math.pow(10,snr/10))
    noise = np.random.normal(0, np.sqrt(noisepower), len(signal))
    return np.array([noise]).T

In [0]:
# y = H*s+w;      
mu = np.zeros(k)
sigma = np.ones((k,k))
snr = 20

noise = awgn(s,snr)
# noise = np.random.normal(loc=mu, scale=sigma, size=(k,k)).view(np.complex128) 
# noise = np.concatenate((np.real(noise), np.imag(noise)), axis=0)



y = H_expanded.dot(s) + noise
print("\nH\n",H_expanded)
print("\ns\n",s)
# print("\nnoise\n",noise)
print("\nnoise \n", noise)
print("\ny\n",y)


H
 [[ 0.66214108 -0.45171898  0.79578875  0.24642896]
 [ 0.38983823  1.23646399  0.29155658  0.51144257]
 [-0.79578875 -0.24642896  0.66214108 -0.45171898]
 [-0.29155658 -0.51144257  0.38983823  1.23646399]]

s
 [[-3]
 [-3]
 [-3]
 [ 3]]

noise 
 [[-0.02214818]
 [-0.35709182]
 [-0.12617904]
 [-0.24729107]]

y
 [[-2.30149384]
 [-4.57634051]
 [-0.34110606]
 [ 4.70158364]]


In [0]:
## CALCULO DE W (variable a minimizar)
w = np.concatenate((s,t,[[1]]))
w_transpose = w.transpose()
print("w\n",w)
print("w_transpose\n",w_transpose)

W = w.dot(w_transpose)
print("\nBig W\n",W)
print("\nBig W\n",W.shape)

w
 [[-3]
 [-3]
 [-3]
 [ 3]
 [ 9]
 [ 9]
 [ 9]
 [ 9]
 [ 1]]
w_transpose
 [[-3 -3 -3  3  9  9  9  9  1]]

Big W
 [[  9   9   9  -9 -27 -27 -27 -27  -3]
 [  9   9   9  -9 -27 -27 -27 -27  -3]
 [  9   9   9  -9 -27 -27 -27 -27  -3]
 [ -9  -9  -9   9  27  27  27  27   3]
 [-27 -27 -27  27  81  81  81  81   9]
 [-27 -27 -27  27  81  81  81  81   9]
 [-27 -27 -27  27  81  81  81  81   9]
 [-27 -27 -27  27  81  81  81  81   9]
 [ -3  -3  -3   3   9   9   9   9   1]]

Big W
 (9, 9)


In [0]:
y


array([[-2.30149384],
       [-4.57634051],
       [-0.34110606],
       [ 4.70158364]])

In [0]:
# Cálculo de la matriz a multiplicar por H

A11 = H_expanded.T.dot(H_expanded) # Check if it is really doing the transpose of H_expanded
A12 = np.zeros((2*k,2*k))
A13 = -H_expanded.T.dot(y) # Check if it is really doing the transpose of H_expanded
A1X = np.column_stack((np.column_stack((A11, A12)), A13))
print("\nFILA 1")
print("\n",A1X)

print("\nFILA 2")
A2X = np.zeros((2*k,A1X.shape[1]))
print("\n",A2X)

print("\nFILA 3")
A31 = (-y.T).dot(H_expanded)
A32= np.zeros((1,2*k))
A33 = (y.T).dot(y)
A3X = np.column_stack((np.column_stack((A31, A32)), A33))
A3X = np.column_stack((A31, A32, A33))
print("\n",A3X)

A = np.concatenate((A1X,A2X,A3X))
print("\n Matriz A")
print("\n",A)
print("\n",A.shape)

# A = [H'*H zeros(2*N,2*N) -H'*y; zeros(2*N,4*N+1); -y'*H zeros(1,2*N) y'*y];



FILA 1

 [[ 1.30868962e+00  5.28139087e-01  3.93100659e-18  3.61524269e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   4.40727538e+00]
 [ 5.28139087e-01  2.05519396e+00 -3.61524269e-01 -1.08260064e-16
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   6.93938340e+00]
 [ 3.93100659e-18 -3.61524269e-01  1.30868962e+00  5.28139087e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.55876838e+00]
 [ 3.61524269e-01 -1.08260064e-16  5.28139087e-01  2.05519396e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -3.05973286e+00]]

FILA 2

 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]

FILA 3

 [[ 4.40727538  6.9393834   1.55876838 -3.05973286  0.          0.
   0.          0.         48.46100847]]

 Matriz A

 [[ 1.30868962e+00  5.28139087e-01  3.93100659e-18  3.61524269e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.000000

In [0]:
import cvxpy as cp
# Remove exponential notation
np.set_printoptions(suppress=True)


# Problem data.
np.random.seed(1)
# print(A)
k = 2 # number of antennas (TX y RX)
zero_array = np.zeros(2*k)
# m = 2
# n = 2
# A = np.random.randn(m, n)
# b = np.random.randn(n,1)
#  2x2 2x1 = 2x1
# c = cp.matmul(A,b)
# print(c)

# Construct the problem.
W = cp.Variable((9,9))

objective = cp.Minimize(cp.trace(cp.matmul(W,A)))
# objective = cp.Minimize(cp.trace(W@A))
constraints = [W >> 0, 
               cp.diag(W[0:2*k, 0:2*k]) - W[2*k:4*k,4*k] == 0,
               cp.diag(W[2*k:4*k, 2*k:4*k]) - 10*W[2*k:4*k,4*k] + 9*np.ones((2*k)) == 0,
               W[4*k, 4*k] == 1 ]

'''
subject to:
1.  W >> 0 (semidefinida positiva)
2. diag(W(1:2*N,1:2*N)) - W(2*N+1:4*N,4*N+1) == 0
'''

prob = cp.Problem(objective, constraints)
prob.solve()

# print("Optimal value", prob.solve())
print("Optimal value of W")
print(W.value) # A numpy ndarray.

WARN: m less than n, problem likely degenerate
Optimal value of W
[[  8.12517554   9.14836878   9.16157447  -8.02743193  -0.00067368
    0.00019167  -0.00104905  -0.00094233  -2.85047124]
 [  9.14836878  10.30040885  10.31527939  -9.03831477  -0.0009475
   -0.0004843   -0.00078089  -0.00126275  -3.20942761]
 [  9.16157447  10.31527939  10.33016997  -9.05136309  -0.00049843
   -0.00068758  -0.00090029  -0.00078496  -3.21406085]
 [ -8.02743193  -9.03831477  -9.05136309   7.93085884   0.00066068
    0.00053993   0.00085595   0.00097845   2.81618068]
 [ -0.00067368  -0.0009475   -0.00049843   0.00066068  72.25175007
    0.00057795  -0.00007759   0.00043051   8.12517528]
 [  0.00019167  -0.0004843   -0.00068758   0.00053993   0.00057795
   94.00408678  -0.00015156   0.00023949  10.30040886]
 [ -0.00104905  -0.00078089  -0.00090029   0.00085595  -0.00007759
   -0.00015156  94.30169877  -0.00001692  10.33017013]
 [ -0.00094233  -0.00126275  -0.00078496   0.00097845   0.00043051
    0.00023949

12.0
